In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
data = pd.read_csv('creditcard.csv')
data.shape

(284807, 31)

In [4]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [41]:
# data = data.sample(frac=.5,random_state=1)
#determine number of fraud cases in dataset
Fraud = data[data['Class']==1]
Valid = data[data['Class']==0]

outlier_fraction = len(Fraud)/ float(len(Valid))
print(outlier_fraction)

print('Fraud case: {}'.format(len(Fraud)))
print('Valid tx: {}'.format(len(Valid)))

0.00143459915612
Fraud case: 51
Valid tx: 35550


In [37]:
data = data.sample(frac=.5,random_state=1)
data.shape

(142404, 31)

In [38]:
# get all columns from dataframe
columns = data.columns.tolist()

columns = [c for c in columns if c not in ['Class']]

target = "Class"

X = data[columns]
Y = data[target]

print(X.shape)
print(Y.shape)

(142404, 30)
(142404L,)


In [23]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
# from sklearn.neighbors import LocalOutlierFactor

In [39]:
state = 1
# define outlier detection tools to be compared
classifier = {
    "Isolation Forest": IsolationForest(max_samples = len(X),
                                        contamination=outlier_fraction,
                                        random_state=state)
#     "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20,
#                             contamination=outlier_fraction)
}

# fit the model
plt.figure(figsize=(9,7))
n_outliers = len(Fraud)

for i,(clf_name,clf) in enumerate(classifier.items()):
    if clf_name == 'Local Outlier Factor':
        y_pred = clf.fit_predict(X)
        scores_pred = clf.negetive_outlier_factor_
    else:   
        clf.fit(X)
        scores_pred = clf.decision_function(X)
        y_pred = clf.predict(X)
        
#     reshape the prediction values to 0 for valid,1 for fraud
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] =1
    
    n_errors = (y_pred != Y).sum()
    
#     run classification metrics
    print('{}: {}'.format(clf_name,n_errors))
    print(accuracy_score(Y,y_pred))
    print(classification_report(Y,y_pred))
        
        


Isolation Forest: 328
0.997696693913
             precision    recall  f1-score   support

          0       1.00      1.00      1.00    142177
          1       0.30      0.32      0.31       227

avg / total       1.00      1.00      1.00    142404

